In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import numpy as np 
import pandas as pd 
import json
import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt
from nltk.tokenize import wordpunct_tokenize
from tqdm import tqdm

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd gdrive/'My Drive'/

/content/gdrive/My Drive


In [0]:
train = pd.read_csv('permutate_anw_ques.csv')

In [0]:
%time
from zipfile import ZipFile

downloaded_train = drive.CreateFile({'id':'10qtOKBsGFQlZ6YzQxV30uHT7truULE1E'}) 
downloaded_train.GetContentFile('train.csv.zip')

with ZipFile('train.csv.zip', 'r') as zipObj:
   zipObj.extractall()

down_main_cat_map = drive.CreateFile({'id': '1l70MFuRBb7IgaVn6hrx0_X3lU0_q0B7C'}) 
down_main_cat_map.GetContentFile('main_category_mapper.json')

downloaded_unsupervised = drive.CreateFile({'id':'1vTeZFDl4ugkRv_6mUCl7gJk8DMnE8ECB'}) 
downloaded_unsupervised.GetContentFile('unsupervised.csv.zip')

with ZipFile('unsupervised.csv.zip', 'r') as zipObj:
   zipObj.extractall()

down_sub_cat_map = drive.CreateFile({'id': '1SJhd1yLU2V6eKGKI_R7Gw0ybGC9RaxJu'}) 
down_sub_cat_map.GetContentFile('sub_category_mapper.json')

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.29 µs


In [0]:
import numpy as np 
import pandas as pd 
import json
import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt
from nltk.tokenize import wordpunct_tokenize
from tqdm import tqdm

In [0]:
train = pd.read_csv('train.csv')

In [0]:
downloaded_sub = drive.CreateFile({'id':'1QqoCIhAJlMxW5_TSaNnOUU08civyHWOP'}) 
downloaded_sub.GetContentFile('test.csv')

In [0]:
test = pd.read_csv('test.csv')
test.head()

,index,question
0,0,"Подскажите сайт российский, вещи заказывать: )..."
1,1,Как заказать Рикардо Милоса в Москве?
2,2,"Пустят ли меня в Израиль, если я работаю в ОАЭ?"
3,3,почему о наличии пальмового масла и консервато...
4,4,Вот во всех бедах России винят правительство.....


In [0]:
import re
def filter_ru_en_words(x):
  x = ' '.join(re.findall('[a-zа-яё0-9]+', x.lower()))
  x = re.sub('[0-9]+', ' число ', x)
  x = re.sub('ё', 'е', x)
  x = re.sub('\s+', ' ', x)
  return x
train['w_cl'] = train.question.apply(filter_ru_en_words)

In [0]:
!pip install pymorphy2

In [0]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
words = [j for i in train.w_cl.tolist() for j in i.split(' ')]
to_norm_form = { i: morph.parse(i)[0].normal_form for i in list(set(words))}

print('Vocab_size :', len(list(set(to_norm_form.values()))))

Vocab_size : 157775


In [0]:
def normalize(x):
  return ' '.join([to_norm_form[i] if i in to_norm_form else i for i in x.split(' ')])

train['norm_question'] = train.w_cl.apply(normalize)

In [0]:
import torch
from torch.utils.data import Dataset, DataLoader

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [0]:
train = train[['main_category', 'example']]
train.columns = ['main_category', 'question']

In [0]:
def process_text(text):
    
    return wordpunct_tokenize(text.lower())
    #text.split(' ')

In [0]:
word2freq = {}

for question in tqdm(train.question):
    
    words = process_text(question)
    
    for word in words:
        
        if word in word2freq:
            word2freq[word] += 1
        else:
            word2freq[word] = 1

100%|██████████| 666666/666666 [00:05<00:00, 130920.69it/s]


In [0]:
%cd ../..

/content


In [0]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz

--2019-12-17 16:47:42--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1306357571 (1.2G) [binary/octet-stream]
Saving to: ‘cc.ru.300.vec.gz’

cc.ru.300.vec.gz    100%[===================>]   1.22G  48.9MB/s    in 25s     

2019-12-17 16:48:08 (49.9 MB/s) - ‘cc.ru.300.vec.gz’ saved [1306357571/1306357571]



In [0]:
!gzip -d cc.ru.300.vec.gz

In [0]:
word2index = {'PAD': 0}
vectors = []
    
word2vec_file = open('cc.ru.300.vec')
    
n_words, embedding_dim = word2vec_file.readline().split()
n_words, embedding_dim = int(n_words), int(embedding_dim)

# Zero vector for PAD
vectors.append(np.zeros((1, embedding_dim)))

progress_bar = tqdm(desc='Read word2vec', total=n_words)

while True:

    line = word2vec_file.readline().strip()

    if not line:
        break
        
    current_parts = line.split()

    current_word = ' '.join(current_parts[:-embedding_dim])

    if current_word in word2freq:

        word2index[current_word] = len(word2index)

        current_vectors = current_parts[-embedding_dim:]
        current_vectors = np.array(list(map(float, current_vectors)))
        current_vectors = np.expand_dims(current_vectors, 0)

        vectors.append(current_vectors)

    progress_bar.update(1)

progress_bar.close()

word2vec_file.close()

vectors = np.concatenate(vectors)

Read word2vec: 100%|██████████| 2000000/2000000 [00:56<00:00, 35713.45it/s]


In [0]:
unk_words = [word for word in word2freq if word not in word2index]
unk_counts = [word2freq[word] for word in unk_words]
n_unk = sum(unk_counts) * 100 / sum(list(word2freq.values()))

sub_sample_unk_words = {word: word2freq[word] for word in unk_words}
sorted_unk_words = list(sorted(sub_sample_unk_words, key=lambda x: sub_sample_unk_words[x], reverse=True))

print('Мы не знаем {:.2f} % слов в датасете'.format(n_unk))
print('Количество неизвестных слов {} из {}, то есть {:.2f} % уникальных слов в словаре'.format(
    len(unk_words), len(word2freq), len(unk_words) * 100 / len(word2freq)))
print('В среднем каждое встречается {:.2f} раз'.format(np.mean(unk_counts)))
print()
print('Топ 5 невошедших слов:')

for i in range(5):
    print(sorted_unk_words[i], 'с количеством вхождениий -', word2freq[sorted_unk_words[i]])

Мы не знаем 3.02 % слов в датасете
Количество неизвестных слов 107889 из 328896, то есть 32.80 % уникальных слов в словаре
В среднем каждое встречается 2.08 раз

Топ 5 невошедших слов:
??? с количеством вхождениий - 8604
!!! с количеством вхождениий - 6976
?) с количеством вхождениий - 6613
?? с количеством вхождениий - 6327
"? с количеством вхождениий - 4581


In [0]:
from torch.utils.data import Dataset
class WordData(Dataset):
    
    def __init__(self, x_data, y_data, word2index, sequence_length=40, pad_token='PAD', verbose=True):
        
        super().__init__()
        
        self.x_data = []
        self.y_data = y_data
        
        self.word2index = word2index
        self.sequence_length = sequence_length
        
        self.pad_token = pad_token
        self.pad_index = self.word2index[self.pad_token]
        
        self.load(x_data, verbose=verbose)
        
    @staticmethod
    def process_text(text):
        
        # Место для вашей предобработки
    
        words = wordpunct_tokenize(text.lower())

        return words
        
    def load(self, data, verbose=True):
        
        data_iterator = tqdm(data, desc='Loading data', disable=not verbose)
        
        for text in data_iterator:
            words = self.process_text(text)
            indexed_words = self.indexing(words)
            self.x_data.append(indexed_words)
    
    def indexing(self, tokenized_text):

        # здесь мы не используем токен UNK, потому что мы мы его специально не учили
        # становится непонятно какой же эмбеддинг присвоить неизвестному слову,
        # поэтому просто выбрасываем наши неизветсные слова
        
        ### CODE ###

        return [self.word2index[token] for token in tokenized_text if token in self.word2index ]
    
    def padding(self, sequence):
        
        # Ограничить длину self.sequence_length
        # если длина меньше максимально - западить
        
        ### CODE ###

        if len(sequence) > self.sequence_length:
            sequence = sequence[:self.sequence_length]
        elif len(sequence) < self.sequence_length:
            sequence = sequence + [self.pad_index] * (self.sequence_length - len(sequence))

        return sequence
    
    def __len__(self):
        
        return len(self.x_data)
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx]
        if len(x)<=self.sequence_length and len(x)>0:
          len_x = len(x)
        elif len(x)<=0:
          len_x=1
        else:
          len_x = self.sequence_length
        
        x = self.padding(x)
        x = torch.Tensor(x).long()
        
        y = self.y_data[idx]
        
        return x, y, len_x

In [0]:
for instance in list(tqdm._instances):
    tqdm._decr_instances(instance)

In [0]:
x_train, x_validation, y_train, y_validation = train_test_split(train.question, train.main_category, test_size=0.1)

train_dataset = WordData(list(x_train), list(y_train), word2index)
train_loader = DataLoader(train_dataset, batch_size=128)

validation_dataset = WordData(list(x_validation), list(y_validation), word2index)
validation_loader = DataLoader(validation_dataset, batch_size=128)


Loading data: 100%|██████████| 66667/66667 [00:00<00:00, 81502.89it/s]


In [0]:
test_dataset = WordData(list(test.question), np.zeros((test.shape[0])), word2index)
test_loader = DataLoader(test_dataset, batch_size=512 )

Loading data: 100%|██████████| 200000/200000 [00:01<00:00, 130887.08it/s]


In [0]:
for x, y, len_x in train_loader:
  print(len_x)
  break

tensor([ 5, 18, 13,  6, 12, 15,  5,  4,  9,  7,  4,  9, 21, 15, 10, 11,  4,  6,
        13,  8,  5,  5, 24, 18, 17,  8, 19,  9,  5, 15,  7, 11, 22,  5,  3,  4,
         4, 22,  7, 13,  9, 10, 10,  8, 14, 11, 10,  8, 22, 22,  6, 24,  9, 11,
        23, 15, 22,  9, 23, 11,  6, 13, 14,  7, 15,  3, 15,  9, 10, 17, 13, 23,
         9,  6, 20,  4,  6, 10, 14,  5, 21,  5, 11, 21, 15, 13, 16,  8,  8, 19,
         8, 13,  5,  5,  6,  5,  4,  8, 15,  7,  5,  5, 10, 22, 20,  9, 12, 18,
        13,  8,  7, 14, 19,  6,  8, 15,  8,  6, 21,  4,  5,  7, 14, 15, 18, 11,
        12, 15])


In [0]:
n_classes = train.main_category.unique().shape[0]

In [0]:
from math import sqrt
class model_with_att(torch.nn.Module):
  def __init__(self, matrix_w, out_GRu_feat, n):
        
        super().__init__()
        
        self.out_GRu_feat = out_GRu_feat
        self.matrix_w = matrix_w

        self.out_biGRu = int(self.out_GRu_feat * 2)
        self.out_CNN = int(self.out_GRu_feat * 3)
        self.n = int(n)

        self.num_embeddings, self.embedding_dim = self.matrix_w.shape
        self.emb_layer = torch.nn.Embedding(self.num_embeddings, self.embedding_dim)
        self.emb_layer.load_state_dict({'weight': torch.tensor(self.matrix_w)})
        self.emb_layer.weight.requires_grad = False

        self.drop_emb = torch.nn.Dropout(0.15)

        self.LSTM = torch.nn.LSTM(self.embedding_dim, out_GRu_feat, num_layers = 2, dropout = 0.25, bidirectional=True, batch_first=True) 

        #attention
        
        self.q_proj = torch.nn.Linear(self.out_biGRu, self.out_biGRu)
        self.k_proj = torch.nn.Linear(self.out_biGRu, self.out_biGRu)
        self.v_proj = torch.nn.Linear(self.out_biGRu, self.out_biGRu)
        self.att_soft = torch.nn.Softmax(dim = 2)

        self.q_proj1 = torch.nn.Linear(self.out_biGRu, self.out_biGRu)
        self.k_proj1 = torch.nn.Linear(self.out_biGRu, self.out_biGRu)
        self.v_proj1 = torch.nn.Linear(self.out_biGRu, self.out_biGRu)
        
        self.permutate = torch.nn.Linear(self.out_biGRu*2, self.out_biGRu)

        self.cnn_3gr = torch.nn.Conv1d(in_channels=int(self.out_biGRu) , out_channels=self.out_GRu_feat, kernel_size=3)
        self.cnn_4gr = torch.nn.Conv1d(in_channels=int(self.out_biGRu) , out_channels=self.out_GRu_feat, kernel_size=4)
        self.cnn_5gr = torch.nn.Conv1d(in_channels=int(self.out_biGRu) , out_channels=self.out_GRu_feat, kernel_size=5)

        self.bn1 = torch.nn.BatchNorm1d(self.out_CNN)
        
        self.linear_1 = torch.nn.Linear(in_features=self.out_CNN, out_features=750)
        gl = sqrt(6./(self.out_CNN+750))
        self.linear_1.weight.data.uniform_(-gl, gl)

        self.relu = torch.nn.ReLU()

        self.drop_out = torch.nn.Dropout(0.1)

        self.bn2 = torch.nn.BatchNorm1d(750)

        self.linear_2 = torch.nn.Linear(in_features=750, out_features=n)
        gl2 = sqrt(6./(750+n))
        self.linear_2.weight.data.uniform_(-gl2, gl2)

        
  def forward(self, x, x_len):
      x_emb = self.emb_layer(x)

      x_emb = self.drop_emb(x_emb)

      x_packed = pack_padded_sequence(x_emb, x_len, batch_first=True, enforce_sorted=False)

      x_lstm, _ = self.LSTM(x_packed)

      x, _ = pad_packed_sequence(x_lstm, batch_first=True)
      
      x_q = self.q_proj(x)
      x_k = self.k_proj(x)
      x_v = self.v_proj(x)

      x_q1 = self.q_proj1(x)
      x_k1 = self.k_proj1(x)
      x_v1 = self.v_proj1(x)

      att_scores = torch.bmm(x_q, x_k.transpose(2,1))/self.out_GRu_feat
      att_dist = self.att_soft(att_scores)
      attention_vectors = torch.bmm(att_dist, x_v)

      att_scores1 = torch.bmm(x_q1, x_k1.transpose(2,1))/self.out_GRu_feat
      att_dist1 = self.att_soft(att_scores1)
      attention_vectors1 = torch.bmm(att_dist1, x_v1)

      x_two_at = torch.cat((attention_vectors, attention_vectors1), dim=-1)

      x_two_at = self.permutate(x_two_at)
      x_gru_s_t = x_two_at.transpose(2,1)

      x_cnn3 = self.cnn_3gr(x_gru_s_t)
      x_cnn4 = self.cnn_4gr(x_gru_s_t)
      x_cnn5 = self.cnn_5gr(x_gru_s_t)

      frst, _ =  x_cnn3.max(dim= -1,)
      sc, _ = x_cnn4.max(dim= -1,)
      thr, _ = x_cnn5.max(dim= -1,)
      
      px = torch.cat((frst, sc, thr), dim=-1)
      
      x = self.bn1(px)
      x = self.linear_1(x)
      x = self.relu(x)
      x = self.drop_out(x)
      x = self.bn2(x)
      x = self.linear_2(x)
    
      return x

In [0]:
model = model_with_att(vectors, 500, n_classes)

In [0]:
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence

In [0]:
max(len_x)

tensor(24)

In [0]:
with torch.no_grad():
    pred = model(x, len_x)

In [0]:
pred.shape

torch.Size([128, 28])

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [0]:
model

model_with_att(
  (emb_layer): Embedding(221008, 300)
  (drop_emb): Dropout(p=0.15, inplace=False)
  (LSTM): LSTM(300, 500, num_layers=2, batch_first=True, dropout=0.25, bidirectional=True)
  (q_proj): Linear(in_features=1000, out_features=1000, bias=True)
  (k_proj): Linear(in_features=1000, out_features=1000, bias=True)
  (v_proj): Linear(in_features=1000, out_features=1000, bias=True)
  (att_soft): Softmax(dim=2)
  (q_proj1): Linear(in_features=1000, out_features=1000, bias=True)
  (k_proj1): Linear(in_features=1000, out_features=1000, bias=True)
  (v_proj1): Linear(in_features=1000, out_features=1000, bias=True)
  (cnn_3gr): Conv1d(2000, 500, kernel_size=(3,), stride=(1,))
  (cnn_4gr): Conv1d(2000, 500, kernel_size=(4,), stride=(1,))
  (cnn_5gr): Conv1d(2000, 500, kernel_size=(5,), stride=(1,))
  (bn1): BatchNorm1d(1500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_1): Linear(in_features=1500, out_features=750, bias=True)
  (relu): ReLU()
  (drop_out): 

In [0]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters())


In [0]:

model = model.to(device)
criterion = criterion.to(device)

In [0]:
from torch.nn.utils.rnn import pack_padded_sequence

epochs = 10
losses = []
best_test_loss = 10.

test_f1 = []

for n_epoch in range(epochs):
    
    train_losses = []
    test_losses = []
    test_targets = []
    test_pred_class = []
    
    progress_bar = tqdm(total=len(train_loader.dataset), desc='Epoch {}'.format(n_epoch + 1))
    
    model.train()
    
    for x, y, x_len,  in train_loader:

    
        x_len = x_len.to(device)
        x = x.to(device)
        y = y.to(device)

        
        optimizer.zero_grad()
        
        pred = model(x, x_len)
        loss = criterion(pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        train_losses.append(loss.item())
        losses.append(loss.item())
        
        progress_bar.set_postfix(train_loss = np.mean(losses[-500:]))

        progress_bar.update(x.shape[0])
        
    progress_bar.close()
    
    model.eval()
    
    for x, y, x_len in validation_loader:
        
        
        x = x.to(device)
        x_len = x_len.to(device)
        y = y.to(device)

        with torch.no_grad():

            pred = model(x, x_len)

            pred = pred.cpu()
            y = y.cpu()

            test_targets.append(y.numpy())
            test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            test_losses.append(loss.item())
        
    mean_test_loss = np.mean(test_losses)

    test_targets = np.concatenate(test_targets).squeeze()
    test_pred_class = np.concatenate(test_pred_class).squeeze()

    f1 = f1_score(test_targets, test_pred_class, average='micro')

    test_f1.append(f1)
    
    print()
    print('Losses: train - {:.3f}, test - {:.3f}'.format(np.mean(train_losses), mean_test_loss))

    print('F1 test - {:.3f}'.format(f1))
        
    # Early stopping:
    if mean_test_loss < best_test_loss:
        best_test_loss = mean_test_loss
    else:
        print('Early stopping')
        break

Epoch 2:   0%|          | 256/599999 [00:00<06:54, 1445.58it/s, train_loss=1.49]


Losses: train - 1.669, test - 1.437
F1 test - 0.586


Epoch 3:   0%|          | 256/599999 [00:00<07:00, 1425.05it/s, train_loss=1.38]


Losses: train - 1.425, test - 1.356
F1 test - 0.609


Epoch 4:   0%|          | 256/599999 [00:00<07:03, 1415.99it/s, train_loss=1.31]


Losses: train - 1.340, test - 1.308
F1 test - 0.621


Epoch 5:   0%|          | 256/599999 [00:00<06:49, 1465.69it/s, train_loss=1.25]


Losses: train - 1.278, test - 1.281
F1 test - 0.629


Epoch 5:  15%|█▌        | 90624/599999 [01:08<06:31, 1300.96it/s, train_loss=1.25]

In [0]:
model.eval()

predictions = []

for x, y, x_len in test_loader:

    x = x.to(device)

    with torch.no_grad():

        pred = model(x,x_len)

        pred = pred.cpu()
        
        predictions.append(np.argmax(pred, axis=1))
        
predictions = np.concatenate(predictions).squeeze()

Epoch 2:   0%|          | 2560/599999 [00:20<05:15, 1895.48it/s, train_loss=1.03]

In [0]:
test['main_category'] = predictions

In [0]:
test = test[['index', 'main_category']]

In [0]:
test.head()

,index,main_category
0,0,15
1,1,15
2,2,25
3,3,17
4,4,12


In [0]:
test.to_csv('submission123.csv', index=False)

In [0]:
from google.colab import files
files.download('submission123.csv')